In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from skimage import io
from skimage import filters
from skimage import transform
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))

ic=io.imread_collection("../input/train/*jpg")
resized=[]
resized3=[]
count=0
for x in ic:
    blur2=filters.gaussian(x)
    resize_2=transform.resize(blur2,(100,100))
    resize_3=transform.resize(x,(100,100))
    resized.append(resize_2)
    resized3.append(resize_3)
    count=count+1;
    if count>3:
        break
print("resized after a gaussian")
io.imshow(resized[0])
plt.show()
plt.plot(resized[0][50])
plt.show()

io.imshow(resized[1])
plt.show()
plt.plot(resized[1][50])
plt.show()

print("resized without a gaussian")
io.imshow(resized3[0])
plt.show()
plt.plot(resized3[0][50])
plt.show()

io.imshow(resized3[1])
plt.show()
plt.plot(resized3[1][50])
plt.show()

# Any results you write to the current directory are saved as output.

kaggle is using the older version of scipy, so anti-aliasing cant be done while resizing ( any other options?) the resize function uses a box filter; tried using a gausian filter before resizing ,the infromation in both the images looks the same, but the ones resized after a gaussian have less noise. so i used the ones after the gaussian filter

splitting the images according to its RGB components and preparing them for a classifier, as in the digits dataset  

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from skimage import io
from skimage import filters
from skimage import transform
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import svm
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output

def read_and_return():
 
 ic=io.imread_collection("../input/train/*jpg")
 #apply gaussian blur, resize and split into rgb componants
 resized_Red=[]
 resized_Blue=[]
 resized_Green=[]
 count=0
 for x in ic:
    blur2=filters.gaussian(x)
    resize_2=transform.resize(blur2,(100,100))
    resize_Red=resize_2[:,:,0]
    resize_Green=resize_2[:,:,1]
    resize_Blue=resize_2[:,:,2]
    resized_Red.append(resize_Red)
    resized_Green.append(resize_Green)
    resized_Blue.append(resize_Blue)
 #display   
 print("resized after a gaussian")
 io.imshow(resized_Red[0])
 plt.show()
 plt.plot(resized_Red[0][50])
 plt.show()

 io.imshow(resized_Red[1])
 plt.show()
 plt.plot(resized_Red[1][50])
 plt.show()
    
 ##get file names
 file_list=ic.files
 lables=pd.read_csv("../input/labels.csv")
 fl_count=0;
 fl_names=[]
 for x in file_list:
    #print(x.replace("/"," "))
    temp=x.replace("/"," ")
    temp1=temp.replace("."," ")
    temp2=temp1.split()
    fl_names.append(temp2[2])
    
 for id_,name in enumerate(lables.id):
    for index,item in enumerate(fl_names):
        if item==name:
            fl_names[index]=lables.breed[id_]
            
 ##prepare the data for a classifier
 resized_Red_=np.array(resized_Red)
 resized_Blue_=np.array(resized_Blue)
 resized_Green_=np.array(resized_Green)
 new_resized_Red=resized_Red_.reshape((len(resized_Red_),-1))
 new_resized_Blue=resized_Blue_.reshape((len(resized_Blue_),-1))
 new_resized_Green=resized_Green_.reshape((len(resized_Green_),-1))
 return new_resized_Red,new_resized_Blue,new_resized_Green,fl_names
(new_resized_Red,new_resized_Blue,new_resized_Green,fl_names)=read_and_return()

print("train_done")

Red_train, Red_test,names_train, names_test = train_test_split(new_resized_Red,fl_names,test_size=0.30, random_state=42)

clif_red=svm.SVC(decision_function_shape='ovo')
clif_red.fit(Red_train,names_train)
#pred=clif_red.predict(Red_test)
score=clif_red.score(Red_test,names_test)
print("Score",score)





#get test data
def Test_data():
  ic=io.imread_collection("../input/test/*jpg")
  resized_Red=[]
  resized_Blue=[]
  resized_Green=[]
  count=0
  for x in ic:
    blur2=filters.gaussian(x)
    resize_2=transform.resize(blur2,(100,100))
    resize_Red=resize_2[:,:,0]
    resize_Green=resize_2[:,:,1]
    resize_Blue=resize_2[:,:,2]
    resized_Red.append(resize_Red)
    resized_Green.append(resize_Green)
    resized_Blue.append(resize_Blue)
  print("test data resized after a gaussian")
  io.imshow(resized_Red[0])
  plt.show()
  plt.plot(resized_Red[0][50])
  plt.show()

  io.imshow(resized_Red[1])
  plt.show()
  plt.plot(resized_Red[1][50])
  plt.show()
  ##prepare the data for a classifier
  resized_Red_=np.array(resized_Red)
  resized_Blue_=np.array(resized_Blue)
  resized_Green_=np.array(resized_Green)
  new_resized_Red=resized_Red_.reshape((len(resized_Red_),-1))
  new_resized_Blue=resized_Blue_.reshape((len(resized_Blue_),-1))
  new_resized_Green=resized_Green_.reshape((len(resized_Green_),-1))
  return new_resized_Red,new_resized_Blue,new_resized_Green
#(test_resized_Red,test_resized_Blue,test_resized_Green)=Test_data()
#print("test_done")
    
##preparing the classifier

the predicted score on the Red part is bad, the plan was to use each spectrum to predict and then using voting to make the final prediction

Running the script on all the colours together below and resizing the image to a smaller size(50,50) to help with processing, but the score didnt change much at all

looking at the differences in spectrum in the graphs. it appears that the spectrums seems to merge on the dog so could we use this difference to isolate dog from the background? the code below tries to do just that. but the images are too different so it doesnt work very well

In [21]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from skimage import io
from skimage import filters
from skimage import transform
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import svm
from PIL import Image
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output

def read_and_return():
 
 ic=io.imread_collection("../input/train/*jpg")
 #apply gaussian blur, resize and split into rgb componants
 resized_Red=[]
 resized_Blue=[]
 resized_Green=[]
 count=0
 for x in ic:
    blur2=filters.gaussian(x,preserve_range=True)
    resize_2=transform.resize(blur2,(100,100))
    resize_Red=resize_2[:,:,0]
    resize_Green=resize_2[:,:,1]
    resize_Blue=resize_2[:,:,2]
    resized_Red.append(resize_Red)
    resized_Green.append(resize_Green)
    resized_Blue.append(resize_Blue)
    count=count+1
    if count==7:
        break
 
 ##isolate dog from the background
 for index_, item_ in enumerate(resized_Red):
  resized_Red_=np.array(resized_Red[index_])
  resized_Blue_=np.array(resized_Blue[index_])
  resized_Green_=np.array(resized_Green[index_])
  new_resized_Red=resized_Red_.ravel()
  new_resized_Blue=resized_Blue_.ravel()
  new_resized_Green=resized_Green_.ravel()
  for index,item in enumerate(new_resized_Red):
   x_y=new_resized_Red[index]-new_resized_Blue[index]
   y_z=new_resized_Blue[index]-new_resized_Green[index]
  #print(x_y,"x_y")
   if x_y<35 and y_z<35:
    continue
   else:
    new_resized_Red[index]=0
    new_resized_Blue[index]=0
    new_resized_Green[index]=0
  rs_Red=new_resized_Red.reshape((100,100))
  rs_Blue=new_resized_Blue.reshape((100,100))
  rs_Green=new_resized_Green.reshape((100,100))
 #print(rs_Red)
  img_red=Image.fromarray(rs_Red)
 #print(img_red)
  plt.imshow(img_red)
  plt.show()
  img_blue=Image.fromarray(rs_Blue)
  img_green=Image.fromarray(rs_Green)
  
a=read_and_return()